In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv("diabetes.csv",index_col = False)
cols = ["Pregnancies","Glucose","BloodPressure","SkinThickness","Insulin","BMI","DiabetesPedigreeFunction","Age"]
data[cols] = data[cols].replace({ 0:np.nan})
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6.0,148.0,72.0,35.0,NaN,33.6,0.627,50.0,1
1,1.0,85.0,66.0,29.0,NaN,26.6,0.351,31.0,0
2,8.0,183.0,64.0,NaN,NaN,23.3,0.672,32.0,1
3,1.0,89.0,66.0,23.0,94.0,28.1,0.167,21.0,0
4,NaN,137.0,40.0,35.0,168.0,43.1,2.288,33.0,1


In [2]:
data.isnull().sum(axis = 0)

Pregnancies                 111
Glucose                       5
BloodPressure                35
SkinThickness               227
Insulin                     374
BMI                          11
DiabetesPedigreeFunction      0
Age                           0
Outcome                       0
dtype: int64

In [3]:
values = data.values
values.shape

(768, 9)

## ChargeUp IMPUTATION

In [4]:
from IMPT import ChargeUp

In [5]:
imputed_x = ChargeUp().fill_matrix(values)

[SoftImpute] Max Singular Value of X_init = 5142.039102
[SoftImpute] Iter 1: observed MAE=2.136221 rank=6
[SoftImpute] Iter 2: observed MAE=2.167593 rank=6
[SoftImpute] Iter 3: observed MAE=2.188064 rank=6
[SoftImpute] Iter 4: observed MAE=2.202709 rank=6
[SoftImpute] Iter 5: observed MAE=2.215269 rank=6
[SoftImpute] Iter 6: observed MAE=2.225528 rank=6
[SoftImpute] Iter 7: observed MAE=2.233842 rank=6
[SoftImpute] Iter 8: observed MAE=2.240267 rank=6
[SoftImpute] Iter 9: observed MAE=2.244892 rank=6
[SoftImpute] Iter 10: observed MAE=2.248227 rank=6
[SoftImpute] Iter 11: observed MAE=2.250602 rank=6
[SoftImpute] Iter 12: observed MAE=2.252134 rank=6
[SoftImpute] Iter 13: observed MAE=2.253029 rank=6
[SoftImpute] Iter 14: observed MAE=2.253581 rank=6
[SoftImpute] Iter 15: observed MAE=2.253977 rank=6
[SoftImpute] Iter 16: observed MAE=2.254220 rank=6
[SoftImpute] Iter 17: observed MAE=2.254357 rank=6
[SoftImpute] Iter 18: observed MAE=2.254391 rank=6
[SoftImpute] Iter 19: observed MAE=

In [6]:
imputed_x 

array([[  6.   , 148.   ,  72.   , ...,   0.627,  50.   ,   1.   ],
       [  1.   ,  85.   ,  66.   , ...,   0.351,  31.   ,   0.   ],
       [  8.   , 183.   ,  64.   , ...,   0.672,  32.   ,   1.   ],
       ...,
       [  5.   , 121.   ,  72.   , ...,   0.245,  30.   ,   0.   ],
       [  1.   , 126.   ,  60.   , ...,   0.349,  47.   ,   1.   ],
       [  1.   ,  93.   ,  70.   , ...,   0.315,  23.   ,   0.   ]])

## Classifier

In [7]:
y = values[:,8]
x = np.delete(imputed_x, 8, 1)   

In [8]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn import datasets
xtr,xt,ytr,yt=train_test_split(x, y, test_size=0.1, random_state=42)

In [9]:
from sklearn import tree

clf = tree.DecisionTreeClassifier()
clf = clf.fit(xtr,ytr)
ytp=clf.predict(xt)

In [10]:
from sklearn.metrics import accuracy_score
print("Decision tree accuracy_score = " , accuracy_score(yt, ytp)*100 , "%")

Decision tree accuracy_score =  66.23376623376623 %


In [11]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(xtr, ytr) 
ytp = neigh.predict(xt)
print("KNN accuracy_score = " , accuracy_score(yt, ytp)*100 , "%")

KNN accuracy_score =  71.42857142857143 %


In [12]:
from sklearn.linear_model import LogisticRegression
log = LogisticRegression()
log.fit(xtr,ytr)
ytp= log.predict(xt)
print("Logistic accuracy_score = " , accuracy_score(yt, ytp)*100 , "%")

Logistic accuracy_score =  72.72727272727273 %


In [13]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(xtr, ytr)
ytp = neigh.predict(xt)
print("KNN accuracy_score = " , accuracy_score(yt, ytp)*100 , "%")

KNN accuracy_score =  71.42857142857143 %


In [14]:
from sklearn import svm
clf = svm.SVC(gamma='scale')
X = np.array(xtr)
y = np.array(ytr)
from sklearn.svm import SVC
clf = SVC(gamma='auto')
clf.fit(X, y)
ytp = clf.predict(xt)
print("SVM accuracy_score = " , accuracy_score(yt, ytp)*100 , "%")

SVM accuracy_score =  64.93506493506493 %


In [15]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Activation
from keras.utils import np_utils
m = Sequential()
m.add(Dense(8, input_shape=(8,)))
m.add(BatchNormalization())
m.add(Activation("relu"))
    
m.add(Dense(6))
m.add(BatchNormalization())
m.add(Activation("relu"))
m.add(Dropout(0.4))
    
m.add(Dense(2, activation="relu"))
    
# output layer
m.add(Dense(1, activation='sigmoid'))

m.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
m.fit(xtr, ytr, nb_epoch=10, batch_size=30)
ytp = m.predict(xt)

/home/phantom/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/phantom/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/10
691/691 [==============================] - 16s 23ms/step - loss: 0.7029 - acc: 0.6165
Epoch 2/10
691/691 [==============================] - 0s 177us/step - loss: 0.6901 - acc: 0.6556
Epoch 3/10
691/691 [==============================] - 0s 182us/step - loss: 0.6923 - acc: 0.6295
Epoch 4/10
691/691 [==============================] - 0s 169us/step - loss: 0.6926 - acc: 0.6744
Epoch 5/10
691/691 [==============================] - 0s 183us/step - loss: 0.6751 - acc: 0.6845
Epoch 6/10
691/691 [==============================] - 0s 179us/step - loss: 0.6804 - acc: 0.6411
Epoch 7/10
691/691 [==============================] - 0s 178us/step - loss: 0.6659 - acc: 0.6802
Epoch 8/10
691/691 [==============================] - 0s 178us/step - loss: 0.6660 - acc: 0.6802
Epoch 9/10
691/691 [==============================] - 0s 173us/step - loss: 0.6654 - acc: 0.6903
Epoch 10/10
691/691 [==============================] - 0s 181us/step - loss: 0.6563 - acc: 0.6932


In [16]:
i=0
for each in ytp: 
    if(each>=0.5):
        ytp[i] = 1
    else:
        ytp[i] = 0
    i+=1
    
print("Deep Learning accuracy_score = " , accuracy_score(yt, ytp)*100 , "%")

Deep Learning accuracy_score =  71.42857142857143 %
